In [1]:
#Load modules
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
from SPARQLWrapper import SPARQLWrapper, N3, JSON
from rdflib import Graph
import plotly.graph_objects as go
import plotly.express as px
import re

#setup some pandas display modes
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 5)
pd.set_option('display.max_colwidth', None)

%matplotlib inline

## Load the data frame

In [2]:
#Read query results from CSV file
get_ratio_df_all_per_global = pd.read_csv("get_ratio_df_all_per_global1.csv")

#drop first column
get_ratio_df_all_per_global = get_ratio_df_all_per_global.drop(['Unnamed: 0'], axis=1)


#change the column to name the facets of the plot
get_ratio_df_all_per_global = get_ratio_df_all_per_global.rename(columns={'merge': 'metricName'})
get_ratio_df_all_per_global['metricName'] = get_ratio_df_all_per_global['metricName'].str.replace('\\','')

get_ratio_df_all_per_global['unique_ds']=get_ratio_df_all_per_global['ds'].astype(str)+', '+get_ratio_df_all_per_global['metricName']
get_ratio_df_all_per_global['unique_task']=get_ratio_df_all_per_global['task'].astype(str)+', '+get_ratio_df_all_per_global['metricName']

get_ratio_df_all_per_global['unique_task_ds_metric']=get_ratio_df_all_per_global['task'].astype(str)+', '+get_ratio_df_all_per_global['ds'].astype(str)+', '+get_ratio_df_all_per_global['metricName']

#plot unique metricName
pd.DataFrame(get_ratio_df_all_per_global["metricName"].unique())

<ipython-input-2-94029eaf80b0>:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  get_ratio_df_all_per_global['metricName'] = get_ratio_df_all_per_global['metricName'].str.replace('\\','')


,0
0,1 in 10 R-at-1
1,F1
...,...
174,DE
175,BA


## Prepare the data frame

In [3]:
#calculate average data frame using 'RATIO' as criteria FOR unique_task
#This graph is fine, however, it will display overlapped datasets used for the same task in the same trajectory.
traj = get_ratio_df_all_per_global.copy()

#task causing some problems
traj = traj.drop(traj[traj["metricName"]=="Parameters"].index)
#traj = traj.drop(traj[traj["unique_task"]=="Natural Language Inference, Parameters"].index)

#Process date field
#traj["date"]=pd.to_datetime(traj["date"]).dt.year

traj["date"] = pd.to_datetime(traj["date"]).dt.strftime('%Y-%m')


In [4]:
traj.head(3)

,ds_count,task,ds,date,model_label,value,percent_of_max_sota,gain,ratio,max_sota,percent_of_max_metric,metricName,unique_ds,unique_task,unique_task_ds_metric
0,1,Sentiment Analysis,1B Words,2012-06,BilSTM,93.0,100.00,93.0,1.00,93.0,1.00,1 in 10 R-at-1,"1B Words, 1 in 10 R-at-1","Sentiment Analysis, 1 in 10 R-at-1","Sentiment Analysis, 1B Words, 1 in 10 R-at-1"
1,1,Citation Intent Classification,ACL-ARC,2013-06,SVM,41.0,60.38,41.0,0.60,67.9,0.42,F1,"ACL-ARC, F1","Citation Intent Classification, F1","Citation Intent Classification, ACL-ARC, F1"
2,1,Citation Intent Classification,ACL-ARC,2016-06,BiLSTM-Attention,51.8,76.29,10.8,0.16,67.9,0.53,F1,"ACL-ARC, F1","Citation Intent Classification, F1","Citation Intent Classification, ACL-ARC, F1"


In [5]:
#How many unique tasks?
len(traj.task.unique())

115

In [6]:
#How many unique benchmarks?
len(traj.ds.unique())

383

In [7]:
#How many unique model_label?
len(traj.model_label.unique())

705

In [8]:
#How many unique metricName?
len(traj.metricName.unique())

175

## Plot using percent_of_max_sota

In [10]:
#calculate average data frame using 'percent_of_max_sota' as criteria FOR unique_task
average_summary = pd.DataFrame(traj.groupby(['unique_task_ds_metric','date'])['percent_of_max_sota'].mean())
average_summary.sort_values(by=['date'], ascending=True)
average_summary.reset_index(inplace=True)  
#Use this is you only want to plot the year, reducing the number of arrows displayed in the graph
#average_summary["date"]=pd.to_datetime(average_summary['date'])
#average_summary["date"]=average_summary["date"].dt.year

#this is a position created to chose which kind of marker will be used in the trajectory
average_summary["in_trajectory"]=1

#iterator to annotate the average_summary df
i=0
#for every task
for t in average_summary.unique_task_ds_metric.unique():
    #reset sota
    sota_per = 0 #(percent_of_max_sota state-of-the-art)
    #here the date can't be unique, because we want to look for the best value per year
    #for every value of a specific task
    for v in average_summary[average_summary["unique_task_ds_metric"]==t].percent_of_max_sota:
        #get the value (variable 'per' - percent_of_max_sota) of this task...
        per = average_summary[(average_summary["unique_task_ds_metric"]==t)&(average_summary["percent_of_max_sota"]==v)].percent_of_max_sota.astype(float)
        #extract the integer
        per = per.iloc[0]
        
        #compare this value with the state-of-the-art
        if(per >= sota_per):
            #print(per)
            sota_per = per #if greater, set this as the new sota
            average_summary.loc[i,"in_trajectory"]= "IN" #set the in_trajectory column as IN
        else:
            average_summary.loc[i,"in_trajectory"]= "OUT" #set the in_trajectory column as IN
            #sota_per = per
        
        #increment the average_summary counter
        i = i+1         


#once the filtering is done, do some formatting
average_summary['percent_of_max_sota'] = average_summary['percent_of_max_sota'].apply(lambda x: round(x, 2))
average_summary = average_summary.rename(columns={'percent_of_max_sota': 'average_perc_max_sota'})
average_summary["date"] = pd.to_datetime(average_summary["date"]).dt.strftime('%Y-%m')

#needs to rename here to average of percentage of maximum sota


###NOW PLOT IT
#try using plotly
import pandas as pd
import plotly.express as px
average_summary_IN = average_summary[average_summary["in_trajectory"]=="IN"]
average_summary_OUT = average_summary[average_summary["in_trajectory"]=="OUT"]

fig = px.line(average_summary_IN, x='date' , y='unique_task_ds_metric' , color='unique_task_ds_metric')
fig.update_traces(line=dict(width=1, color='black'))


#this trace adds first all the dates with a data point attached to it
#fig3_df=average_summary[average_summary["in_trajectory"]=="OUT"]
#fig3_df["date"]=pd.to_datetime(fig3_df['date'])
#fig3_df["date"]=fig3_df["date"].dt.year

fig.add_trace(go.Scatter(
            x = average_summary_OUT["date"],
            y = average_summary_OUT["unique_task_ds_metric"],
            mode='markers',
            name=None,
            marker=dict(
                size=15, 
                #symbol=48,
                color="white",
                opacity=1,
                ),
            hovertemplate=average_summary_OUT["unique_task_ds_metric"]+
                  "<BR>Model_label: "+ average_summary_OUT["unique_task_ds_metric"]+
                  "<BR>average_perc_max_sota: "+ average_summary_OUT["average_perc_max_sota"].astype("string"),
    
        ))

        
#then as use the average_summary df to add only the data that forms a trajectory#
#this value/date is the average of the percentual of maximum value achieved for the metrics at that date.



fig.add_trace(go.Scatter(
    x = average_summary_IN["date"],
    y = average_summary_IN["unique_task_ds_metric"],
    mode='markers',
    name=None,
    hovertemplate=average_summary_IN["unique_task_ds_metric"]+
                  "<BR>Model_label: "+ average_summary_IN["unique_task_ds_metric"]+
                  "<BR>average_perc_max_sota: "+ average_summary_IN["average_perc_max_sota"].astype("string"),
    marker=dict(
        size=15, #alpha ratio
        symbol=48, #https://plotly.com/python/marker-style/
        opacity=0.7,
        color=average_summary_IN["average_perc_max_sota"], #set color equal to a variable
        colorscale='Viridis', # one of plotly colorscales
        colorbar=dict(title='average_perc_max_sota', lenmode="pixels", len=2500),
        showscale=True
    )
))



fig.update_traces(marker=dict(line=dict(color='black',width=1)))

fig.update_xaxes(showgrid=True, gridcolor="lightBlue", title="Year")
fig.update_yaxes(showgrid=True, gridcolor="lightBlue", title="unique_task_ds_metric")

fig.update_layout(title="Trajectory for ratio (unique_task_ds_metric per year)", 
                  showlegend=False,
                  plot_bgcolor='white',
                  height=16000,
                  width=1300,
                  xaxis = dict(
                    #tickmode = 'linear',
                    #tick0 = 2012,
                    #dtick = 1
                    )
                 ) # set the background colour)

fig.show()